Primer paso es leer BD del servidor donde se tenga la tabla de los pesometros antiguos con problemas.
En este caso es el servidor de google

In [14]:
import mysql.connector
import time
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter


db = mysql.connector.connect(host="35.198.8.176",user="alvarov",passwd="lata5comprar",db="bsa_loblanco",charset = "utf8" )        # el nombre de la base de datos
#analisisBDInaplast
cur = db.cursor()
print("conexion a gc")
consulta="SHOW TABLES"
cur.execute(consulta)
for row in cur.fetchall():
    print (str(row[0]))
print (" ")


conexion a gc
camiones_reg
cargador_reg
pesometroreparado
pesometros
 


Los datos en bruto estan en la tabla pesometro columna acc1 pesometro integral y acc2 pesometro grava 3/4.
row[1]=fecha, row[5]=integral, row[6]=grava3/4

In [15]:
consulta="DESCRIBE pesometros"
cur.execute(consulta)
for row in cur.fetchall():
    print (str(row[0]))
print (" ")


id
fecha
tph1
tph2
tph3
acc1
acc2
acc3
diff1
diff2
diff3
 


vamos a leer el primer y ultimo registro de la tabla pesometro, en la consulta anterior se muestran los nombre de las columnas


In [13]:
integralx=[]
gravax=[]
fechaIni="'2019-08-01 08:00:03'"
fechaFin="'2019-08-20 08:00:00'" 
#consula
tabla="pesometros"
consulta="SELECT * FROM "+tabla+" WHERE fecha >= "+fechaIni+ "ORDER BY id ASC LIMIT 1"
print(consulta)
cur.execute(consulta)
print ("El primer registro de la consulta ")
for row in cur.fetchall():
     print (str(row[0]) + " " +str(row[1])+" "+str(row[2]) + " " +str(row[3]))
     integralx.append(row[2])
     gravax.append(row[3])
print (" ")
print ("El ultimo registro de la consulta ")
tabla="pesometros"
consulta="SELECT * FROM "+tabla+" WHERE fecha <= "+fechaFin+ "ORDER BY id DESC LIMIT 10"
cur.execute(consulta)
for row in cur.fetchall():
     print (str(row[0]) + " " +str(row[1])+" "+str(row[2]) + " " +str(row[3]))
     integralx.append(row[2])
     gravax.append(row[3])
print (" ")

SELECT * FROM pesometros WHERE fecha >= '2019-08-01 08:00:03'ORDER BY id ASC LIMIT 1
El primer registro de la consulta 
54159986 2019-08-07 19:20:13 -1 -1
 
El ultimo registro de la consulta 
54312212 2019-08-16 15:46:44 240 120
54312211 2019-08-16 15:46:41 240 120
54312210 2019-08-16 15:46:38 240 120
54312209 2019-08-16 15:46:35 240 120
54312208 2019-08-16 15:46:32 240 120
54312207 2019-08-16 15:46:29 260 120
54312206 2019-08-16 15:46:26 260 100
54312205 2019-08-16 15:46:23 260 100
54312204 2019-08-16 15:46:20 260 100
54312203 2019-08-16 15:46:17 260 100
 
